In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%tensorflow_version 2.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

import math
import logging
logger = tf.get_logger()
logger.setLevel((logging.ERROR))

In [5]:
dataset,metadata = tfds.load('fashion_mnist', as_supervised=True,with_info=True)
train_dataset,test_dataset=dataset['train'],dataset['test']

Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.0.incomplete6XBMOP/fashion_mnist-train.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.0.incomplete6XBMOP/fashion_mnist-test.tfrecord
Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.0. Subsequent calls will reuse this data.


In [0]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']

In [7]:
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(num_train_examples))
print("Number of test examples:     {}".format(num_test_examples))

Number of training examples: 60000
Number of test examples:     10000


In [0]:
def normalize(images,labels):
  images = tf.cast(images,tf.float32)
  images /=255
  return images,labels

train_dataset = train_dataset.map(normalize)
test_dataset = test_dataset.map(normalize)

train_dataset = train_dataset.cache()
test_dataset = test_dataset.cache()

In [0]:
model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32,(3,3),padding='same',activation=tf.nn.relu,input_shape=(28,28,1)),
        tf.keras.layers.MaxPooling2D((2,2),strides=2),
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation=tf.nn.relu),
        tf.keras.layers.MaxPooling2D((2,2),strides=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation=tf.nn.relu),
        tf.keras.layers.Dense(10)                     
])

In [0]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [0]:
BATCH_SIZE = 32
train_dataset = train_dataset.repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [14]:
model.fit(train_dataset, epochs=10, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

Train on 1875 steps
Epoch 1/10
1875/1875 [==============================] - 29s 15ms/step - loss: 0.3966 - acc: 0.8561
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2569 - acc: 0.9058
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2155 - acc: 0.9206
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1797 - acc: 0.9334
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1583 - acc: 0.9419
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1312 - acc: 0.9512
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1136 - acc: 0.9580
Epoch 8/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0951 - acc: 0.9646
Epoch 9/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0796 - acc: 0.9708
Epoch 10/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0673 - acc: 0.9747


In [15]:
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples/32))
print('Accuracy on test dataset:', test_accuracy)

313/313 [==============================] - 4s 14ms/step - loss: 0.3598 - acc: 0.9165
Accuracy on test dataset: 0.9165
